## Session 1.5: Pizza Service Dialoge (Taskmaster-1, übersetzt für EMNLP 2020)

Dataset from M Amin Farajian, António V Lopes, André FT Martins, Sameen Maruf, Gholamreza Haffari (2020). Findings of the wmt 2020 shared task on chat translation (https://aclanthology.org/2020.wmt-1.3/)

Wir bekommen den Auftrag ein Dialogsystem für einen Pizza Service zu erstellen, um automatisiert Anfragen bearbeiten zu können. Nun stellt sich die gute Frage: wie reden die Menschen in einem entsprechenden Dialog? Ein guter Ansatz ist zuerst bestehende Dialoge anzuschauen und Herausforderungen (wie z.B anaphorische Referenzen) zu identifizieren.

Ziele der heutigen Übung:
- Einen bestehenden Datensatz runterladen
- Dialoge analysieren und verstehen

### Vorverarbeitung des Datensatzes

Um zu wissen womit wir es zu tun haben, lesen wir zuerst den Abstract des Papers (Farajian, 2020) und die README des GitHub Repos: https://github.com/Unbabel/BConTrasT


Zuerst laden wir den Datensatz mit git herunter:

In [ ]:
!git clone --depth 1 https://github.com/Unbabel/BConTrasT ../data/BConTrasT

Da wir mit dem Datensatz nichts trainieren möchten, benötigen wir kein train-test split, sondern laden uns alle vorhandenen Dialoge in ein pandas dataframe:

In [ ]:
import json
import os
import pandas as pd

DATASET_FOLDER = f'..{os.path.sep}data{os.path.sep}BConTrasT'
DATASET_FILES = ['train.json', 'dev.json', 'test1_corpus-with_gold_references.json']

dict_dataset = {}
for ds_file in DATASET_FILES:
    with open(os.path.join(DATASET_FOLDER, ds_file), encoding="utf-8") as file:
        dict_dataset.update(json.load(file))

dict_dataset

Schau dir die Daten an, wie viele Dialoge sind im Datensatz?

In [ ]:
print(f'Number of dialogs: {len(dict_dataset)}')

Es handelt sich um einen multilingualen Datensatz, in welchem die jeweiligen Äußerungen in Source und Target mit unterschiedlicher Sprache gemischt sind. Lasst uns den Datensatz etwas umformen, so dass wir ihn für die weitere Verarbeitung praktisch in einem Pandas Dataframe vorliegen haben:

In [ ]:
utterance_list = []
for dialogID, dialog_steps in dict_dataset.items():
    for step in dialog_steps:
        dict1 = {
            'dialogID': dialogID, 
            'utteranceID': step['utteranceID'],
            'speaker': step['speaker'],
            'de': step['source'] if step['speaker'] == 'customer' else step['target'],
            'en': step['target'] if step['speaker'] == 'customer' else step['source']
        }
        utterance_list.append(dict1)

# to flatten the data we create a list of dicts for each row and add this list en-bloc to the pandas dataframe. Using the append or concat function of pandas might work but is very inefficient as the data is copied for each call. 
df = pd.DataFrame(utterance_list)
df.head(5)

Im Datensatz sind nun jedoch nicht nur Pizza Dialoge. Füge eine Spalte ```pizza_domain``` hinzu, die klassifiziert ob die jeweilige Äußerung das Wort ```Pizza``` enthält. Wenn dem so ist soll der Wert ```True``` sein, ansonsten ```False```

In [ ]:
# adding a column if the dialog step contains the word pizza or Pizza
df['pizza_domain'] = df['de'].apply(lambda x: 'pizza' in x.lower())

Wir können davon ausgehen, dass sofern eine Äußerung das Wort Pizza enthält der gesamte Dialog in der Domäne ist. Schreibe Python Code, der dir alle DialogIDs mit Pizza Service Dialogen ausgibt. 

Tipp: drop_duplicates() könnte hilfreich sein ;-)

In [ ]:
dialog_ids_pizza_domain = list(df[df['pizza_domain']]['dialogID'].drop_duplicates())
dialog_ids_pizza_domain

Und damit haben wir den Datensatz entsprechend aufbereitet, so dass wir damit einen Eindruck für unseren Pizza Dialog bekommen können. Wir können nun auf jeden Dialog folgendermaßen zugreifen:

In [ ]:
df[df['dialogID'] == dialog_ids_pizza_domain[1]]

### Analyse des Datensatzes

Schaue dir nun 3-5 verschiedene Dialoge an. Identifiziere Äußerungen die folgende Dialogeigenschaften enthalten:
- implizite Verifikation 
- explizite Verifikation
- anaphorische Referenz
- Missverständnis
- Nichtverständnis
